In [85]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

# data

## diff 만들기 > ratings 생성

In [86]:
import pickle
with open('./processed/country_to_idx.pkl', 'rb') as f:
    country_to_idx = pickle.load(f)

with open('./processed/idx_to_country.pkl', 'rb') as f:
    idx_to_country = pickle.load(f)

In [87]:
merge_all = pd.read_csv('./processed/merge_all.csv')
merge_all

,country,year,item_id,수출금액,GDP,preference,WTI,USD,JPY,EUR,CNY,diff
0,Afghanistan,2013,10.0,1.0,2.056449e+07,4.862752e-08,97.6075,1094.97,1123.73,1453.47,178.11,0
1,Afghanistan,2013,16.0,13.0,2.056449e+07,6.321578e-07,97.6075,1094.97,1123.73,1453.47,178.11,0
2,Afghanistan,2013,19.0,37.0,2.056449e+07,1.799218e-06,97.6075,1094.97,1123.73,1453.47,178.11,1
3,Afghanistan,2013,20.0,1.0,2.056449e+07,4.862752e-08,97.6075,1094.97,1123.73,1453.47,178.11,0
4,Afghanistan,2013,21.0,142.0,2.056449e+07,6.905108e-06,97.6075,1094.97,1123.73,1453.47,178.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...
105364,Zimbabwe,2021,87.0,1289.0,2.837124e+07,4.543334e-05,68.2225,1145.07,1041.92,1353.40,177.56,1
105365,Zimbabwe,2021,90.0,322.0,2.837124e+07,1.134952e-05,68.2225,1145.07,1041.92,1353.40,177.56,0
105366,Zimbabwe,2021,94.0,1.0,2.837124e+07,3.524696e-08,68.2225,1145.07,1041.92,1353.40,177.56,1
105367,Zimbabwe,2021,95.0,13.0,2.837124e+07,4.582105e-07,68.2225,1145.07,1041.92,1353.40,177.56,1


In [88]:
# country_id를 고유번호로 바꾸기
merge_all['country'] = merge_all['country'].map(country_to_idx)

merge_all

,country,year,item_id,수출금액,GDP,preference,WTI,USD,JPY,EUR,CNY,diff
0,0,2013,10.0,1.0,2.056449e+07,4.862752e-08,97.6075,1094.97,1123.73,1453.47,178.11,0
1,0,2013,16.0,13.0,2.056449e+07,6.321578e-07,97.6075,1094.97,1123.73,1453.47,178.11,0
2,0,2013,19.0,37.0,2.056449e+07,1.799218e-06,97.6075,1094.97,1123.73,1453.47,178.11,1
3,0,2013,20.0,1.0,2.056449e+07,4.862752e-08,97.6075,1094.97,1123.73,1453.47,178.11,0
4,0,2013,21.0,142.0,2.056449e+07,6.905108e-06,97.6075,1094.97,1123.73,1453.47,178.11,0
...,...,...,...,...,...,...,...,...,...,...,...,...
105364,202,2021,87.0,1289.0,2.837124e+07,4.543334e-05,68.2225,1145.07,1041.92,1353.40,177.56,1
105365,202,2021,90.0,322.0,2.837124e+07,1.134952e-05,68.2225,1145.07,1041.92,1353.40,177.56,0
105366,202,2021,94.0,1.0,2.837124e+07,3.524696e-08,68.2225,1145.07,1041.92,1353.40,177.56,1
105367,202,2021,95.0,13.0,2.837124e+07,4.582105e-07,68.2225,1145.07,1041.92,1353.40,177.56,1


In [89]:
# pickle 읽기
with open('./processed/export_GDP_pop_fx_wti.pkl', 'rb') as f:
    data = pd.read_pickle(f)

In [90]:
data

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI
0,2012,Afghanistan,3.0,1.0,2.020357e+07,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
1,2012,Afghanistan,4.0,0.0,2.020357e+07,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
2,2012,Afghanistan,7.0,2.0,2.020357e+07,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
3,2012,Afghanistan,8.0,0.0,2.020357e+07,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
4,2012,Afghanistan,10.0,12.0,2.020357e+07,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116390,2021,Zimbabwe,87.0,1289.0,2.837124e+07,4.543334e-05,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.222500
116391,2021,Zimbabwe,90.0,322.0,2.837124e+07,1.134952e-05,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.222500
116392,2021,Zimbabwe,94.0,1.0,2.837124e+07,3.524696e-08,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.222500
116393,2021,Zimbabwe,95.0,13.0,2.837124e+07,4.582105e-07,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.222500


In [91]:
# country_id를 고유번호로 바꾸기
data['country'] = data['country'].map(country_to_idx)

In [92]:
data

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI
0,2012,0,3.0,1.0,2.020357e+07,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
1,2012,0,4.0,0.0,2.020357e+07,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
2,2012,0,7.0,2.0,2.020357e+07,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
3,2012,0,8.0,0.0,2.020357e+07,0.000000e+00,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
4,2012,0,10.0,12.0,2.020357e+07,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116390,2021,202,87.0,1289.0,2.837124e+07,4.543334e-05,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.222500
116391,2021,202,90.0,322.0,2.837124e+07,1.134952e-05,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.222500
116392,2021,202,94.0,1.0,2.837124e+07,3.524696e-08,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.222500
116393,2021,202,95.0,13.0,2.837124e+07,4.582105e-07,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.222500


In [93]:
data = data[data['year'] != 2012]
data

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI
11026,2013,0,10.0,1.0,2.056449e+07,4.862752e-08,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075
11027,2013,0,16.0,13.0,2.056449e+07,6.321578e-07,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075
11028,2013,0,19.0,37.0,2.056449e+07,1.799218e-06,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075
11029,2013,0,20.0,1.0,2.056449e+07,4.862752e-08,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075
11030,2013,0,21.0,142.0,2.056449e+07,6.905108e-06,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116390,2021,202,87.0,1289.0,2.837124e+07,4.543334e-05,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.2225
116391,2021,202,90.0,322.0,2.837124e+07,1.134952e-05,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.2225
116392,2021,202,94.0,1.0,2.837124e+07,3.524696e-08,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.2225
116393,2021,202,95.0,13.0,2.837124e+07,4.582105e-07,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.2225


In [94]:
data.rename(columns={'품목코드':'item_id'}, inplace=True)
data.head(3)

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_33589/3268127780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'품목코드':'item_id'}, inplace=True)


,year,country,item_id,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI
11026,2013,0,10.0,1.0,2.056449e+07,4.862752e-08,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075
11027,2013,0,16.0,13.0,2.056449e+07,6.321578e-07,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075
11028,2013,0,19.0,37.0,2.056449e+07,1.799218e-06,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075


In [95]:
merge_all.head(3)

,country,year,item_id,수출금액,GDP,preference,WTI,USD,JPY,EUR,CNY,diff
0,0,2013,10.0,1.0,2.056449e+07,4.862752e-08,97.6075,1094.97,1123.73,1453.47,178.11,0
1,0,2013,16.0,13.0,2.056449e+07,6.321578e-07,97.6075,1094.97,1123.73,1453.47,178.11,0
2,0,2013,19.0,37.0,2.056449e+07,1.799218e-06,97.6075,1094.97,1123.73,1453.47,178.11,1


In [96]:
# merge_all=diff
ratings = pd.merge(data, merge_all, how='left', on=['country','year','item_id','GDP','preference','WTI','USD','JPY','EUR','CNY','수출금액'])
ratings

,year,country,item_id,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,1.0,2.056449e+07,4.862752e-08,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075,NaN
1,2013,0,16.0,13.0,2.056449e+07,6.321578e-07,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075,0.0
2,2013,0,19.0,37.0,2.056449e+07,1.799218e-06,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075,1.0
3,2013,0,20.0,1.0,2.056449e+07,4.862752e-08,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075,NaN
4,2013,0,21.0,142.0,2.056449e+07,6.905108e-06,0.017864,31541209.0,0.035276,1094.97,1123.73,1453.47,178.11,97.6075,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105364,2021,202,87.0,1289.0,2.837124e+07,4.543334e-05,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.2225,1.0
105365,2021,202,90.0,322.0,2.837124e+07,1.134952e-05,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.2225,0.0
105366,2021,202,94.0,1.0,2.837124e+07,3.524696e-08,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.2225,1.0
105367,2021,202,95.0,13.0,2.837124e+07,4.582105e-07,0.318998,15993524.0,0.020668,1145.07,1041.92,1353.40,177.56,68.2225,NaN


In [97]:
ratings.describe()

,year,country,item_id,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,diff
count,105369.000000,105369.000000,105369.000000,1.053690e+05,1.053690e+05,1.053690e+05,105369.000000,1.053690e+05,105369.000000,105369.000000,105369.000000,105369.000000,105369.000000,105369.000000,77736.000000
mean,2017.053384,103.874346,50.803253,4.694279e+04,6.404606e+08,1.920689e-04,0.038022,5.480439e+07,0.011590,1129.877052,1038.781895,1330.044017,172.647963,62.373381,0.482363
std,2.581833,57.730341,27.332500,7.347440e+05,2.209886e+09,3.325642e-03,0.295688,1.777078e+08,0.013928,37.850792,56.245318,59.711240,4.600655,19.018039,0.499692
min,2013.000000,0.000000,1.000000,0.000000e+00,3.681166e+04,0.000000e+00,-0.999033,1.069400e+04,-0.041673,1053.300000,935.060000,1255.780000,166.480000,38.595833,0.000000
25%,2015.000000,55.000000,28.000000,2.000000e+00,1.898180e+07,3.418803e-08,-0.016736,4.026336e+06,0.002953,1101.080000,997.070000,1284.100000,168.680000,49.307500,0.000000
50%,2017.000000,102.000000,51.000000,1.050000e+02,7.893026e+07,1.385968e-06,0.037601,1.075468e+07,0.010646,1132.100000,1041.920000,1305.860000,170.950000,57.099167,0.000000
75%,2019.000000,151.000000,73.000000,1.997000e+03,3.750000e+08,1.713603e-05,0.088785,3.801174e+07,0.019160,1161.110000,1070.630000,1353.400000,177.560000,68.222500,1.000000
max,2021.000000,202.000000,99.000000,6.793586e+07,2.330000e+10,3.165603e-01,8.974569,1.412360e+09,0.225784,1179.900000,1123.730000,1453.470000,179.520000,97.607500,1.000000


In [98]:
# 2012년도 USD로 나누기
data = ratings.iloc[0]['USD']
ratings['USD'] =ratings['USD'] / data
ratings['JPY'] =ratings['JPY'] / data
ratings['EUR'] =ratings['EUR'] / data
ratings['CNY'] =ratings['CNY'] / data

In [99]:
#min-max 정규화
ratings['WTI'] = (ratings['WTI'] - ratings['WTI'].min()) / (ratings['WTI'].max() - ratings['WTI'].min())
ratings

,year,country,item_id,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,1.0,2.056449e+07,4.862752e-08,0.017864,31541209.0,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,NaN
1,2013,0,16.0,13.0,2.056449e+07,6.321578e-07,0.017864,31541209.0,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
2,2013,0,19.0,37.0,2.056449e+07,1.799218e-06,0.017864,31541209.0,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,1.0
3,2013,0,20.0,1.0,2.056449e+07,4.862752e-08,0.017864,31541209.0,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,NaN
4,2013,0,21.0,142.0,2.056449e+07,6.905108e-06,0.017864,31541209.0,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105364,2021,202,87.0,1289.0,2.837124e+07,4.543334e-05,0.318998,15993524.0,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0
105365,2021,202,90.0,322.0,2.837124e+07,1.134952e-05,0.318998,15993524.0,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,0.0
105366,2021,202,94.0,1.0,2.837124e+07,3.524696e-08,0.318998,15993524.0,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0
105367,2021,202,95.0,13.0,2.837124e+07,4.582105e-07,0.318998,15993524.0,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,NaN


In [100]:
ratings.describe()

,year,country,item_id,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,diff
count,105369.000000,105369.000000,105369.000000,1.053690e+05,1.053690e+05,1.053690e+05,105369.000000,1.053690e+05,105369.000000,105369.000000,105369.000000,105369.000000,105369.000000,105369.000000,77736.000000
mean,2017.053384,103.874346,50.803253,4.694279e+04,6.404606e+08,1.920689e-04,0.038022,5.480439e+07,0.011590,1.031879,0.948685,1.214685,0.157674,0.402930,0.482363
std,2.581833,57.730341,27.332500,7.347440e+05,2.209886e+09,3.325642e-03,0.295688,1.777078e+08,0.013928,0.034568,0.051367,0.054532,0.004202,0.322276,0.499692
min,2013.000000,0.000000,1.000000,0.000000e+00,3.681166e+04,0.000000e+00,-0.999033,1.069400e+04,-0.041673,0.961944,0.853959,1.146862,0.152041,0.000000,0.000000
25%,2015.000000,55.000000,28.000000,2.000000e+00,1.898180e+07,3.418803e-08,-0.016736,4.026336e+06,0.002953,1.005580,0.910591,1.172726,0.154050,0.181518,0.000000
50%,2017.000000,102.000000,51.000000,1.050000e+02,7.893026e+07,1.385968e-06,0.037601,1.075468e+07,0.010646,1.033910,0.951551,1.192599,0.156123,0.313554,0.000000
75%,2019.000000,151.000000,73.000000,1.997000e+03,3.750000e+08,1.713603e-05,0.088785,3.801174e+07,0.019160,1.060403,0.977771,1.236016,0.162160,0.502048,1.000000
max,2021.000000,202.000000,99.000000,6.793586e+07,2.330000e+10,3.165603e-01,8.974569,1.412360e+09,0.225784,1.077564,1.026266,1.327406,0.163950,1.000000,1.000000


In [101]:
ratings = ratings[['year','country','item_id','preference','GDP_growth','pop_growth','USD','JPY','EUR','CNY','WTI','diff']]
ratings

,year,country,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,4.862752e-08,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,NaN
1,2013,0,16.0,6.321578e-07,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
2,2013,0,19.0,1.799218e-06,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,1.0
3,2013,0,20.0,4.862752e-08,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,NaN
4,2013,0,21.0,6.905108e-06,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
105364,2021,202,87.0,4.543334e-05,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0
105365,2021,202,90.0,1.134952e-05,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,0.0
105366,2021,202,94.0,3.524696e-08,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0
105367,2021,202,95.0,4.582105e-07,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,NaN


In [102]:
ratings.rename(columns={'country':'country_id'}, inplace=True)

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_33589/783160813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings.rename(columns={'country':'country_id'}, inplace=True)


In [103]:
ratings

,year,country_id,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,4.862752e-08,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,NaN
1,2013,0,16.0,6.321578e-07,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
2,2013,0,19.0,1.799218e-06,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,1.0
3,2013,0,20.0,4.862752e-08,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,NaN
4,2013,0,21.0,6.905108e-06,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
105364,2021,202,87.0,4.543334e-05,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0
105365,2021,202,90.0,1.134952e-05,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,0.0
105366,2021,202,94.0,3.524696e-08,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0
105367,2021,202,95.0,4.582105e-07,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,NaN


In [104]:
ratings.isnull().sum()

year              0
country_id        0
item_id           0
preference        0
GDP_growth        0
pop_growth        0
USD               0
JPY               0
EUR               0
CNY               0
WTI               0
diff          27633
dtype: int64

## test > CF에서 가져온 것들

In [105]:
test_set = pd.read_csv('./processed/test13_18_new.csv')
test_set

,country_id,year,item_id,GDP_growth,pop_growth,preference
0,0,2013,1.0,0.101959,0.287706,1.108999e-08
1,0,2013,2.0,0.101959,0.287706,6.544139e-07
2,0,2013,3.0,0.101959,0.287706,1.969499e-06
3,0,2013,4.0,0.101959,0.287706,7.394813e-08
4,0,2013,5.0,0.101959,0.287706,3.336427e-07
...,...,...,...,...,...,...
64884,202,2018,80.0,0.194652,0.232127,7.380276e-07
64885,202,2018,86.0,0.194652,0.232127,6.606483e-07
64886,202,2018,88.0,0.194652,0.232127,2.919775e-06
64887,202,2018,89.0,0.194652,0.232127,8.232880e-06


In [106]:
ratings

,year,country_id,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,4.862752e-08,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,NaN
1,2013,0,16.0,6.321578e-07,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
2,2013,0,19.0,1.799218e-06,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,1.0
3,2013,0,20.0,4.862752e-08,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,NaN
4,2013,0,21.0,6.905108e-06,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
105364,2021,202,87.0,4.543334e-05,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0
105365,2021,202,90.0,1.134952e-05,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,0.0
105366,2021,202,94.0,3.524696e-08,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0
105367,2021,202,95.0,4.582105e-07,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,NaN


In [66]:
# data 합치기
ratings = pd.concat([ratings, test_set])
ratings.reset_index(drop=True, inplace=True)
ratings

,year,country_id,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,4.862752e-08,0.017864,0.035276,1.0,1.026266,1.327406,0.162662,1.0,NaN
1,2013,0,16.0,6.321578e-07,0.017864,0.035276,1.0,1.026266,1.327406,0.162662,1.0,0.0
2,2013,0,19.0,1.799218e-06,0.017864,0.035276,1.0,1.026266,1.327406,0.162662,1.0,1.0
3,2013,0,20.0,4.862752e-08,0.017864,0.035276,1.0,1.026266,1.327406,0.162662,1.0,NaN
4,2013,0,21.0,6.905108e-06,0.017864,0.035276,1.0,1.026266,1.327406,0.162662,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
170253,2018,202,80.0,7.380276e-07,0.194652,0.232127,NaN,NaN,NaN,NaN,NaN,NaN
170254,2018,202,86.0,6.606483e-07,0.194652,0.232127,NaN,NaN,NaN,NaN,NaN,NaN
170255,2018,202,88.0,2.919775e-06,0.194652,0.232127,NaN,NaN,NaN,NaN,NaN,NaN
170256,2018,202,89.0,8.232880e-06,0.194652,0.232127,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
from tqdm import tqdm

# USD/JPY/EUR/CNY/WTI 채워넣기
for i in tqdm(range(2013,2019)):
    ratings.loc[(ratings['year']==i) & (ratings['USD'].isnull()), 'USD'] = ratings[ratings['year']==i]['USD'].unique()[0]
    ratings.loc[(ratings['year']==i) & (ratings['JPY'].isnull()), 'JPY'] = ratings[ratings['year']==i]['JPY'].unique()[0]
    ratings.loc[(ratings['year']==i) & (ratings['EUR'].isnull()), 'EUR'] = ratings[ratings['year']==i]['EUR'].unique()[0]
    ratings.loc[(ratings['year']==i) & (ratings['CNY'].isnull()), 'CNY'] = ratings[ratings['year']==i]['CNY'].unique()[0]
    ratings.loc[(ratings['year']==i) & (ratings['WTI'].isnull()), 'WTI'] = ratings[ratings['year']==i]['WTI'].unique()[0]
    

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 121.88it/s]


In [68]:
ratings

,year,country_id,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,4.862752e-08,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,NaN
1,2013,0,16.0,6.321578e-07,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.0
2,2013,0,19.0,1.799218e-06,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,1.0
3,2013,0,20.0,4.862752e-08,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,NaN
4,2013,0,21.0,6.905108e-06,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
170253,2018,202,80.0,7.380276e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN
170254,2018,202,86.0,6.606483e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN
170255,2018,202,88.0,2.919775e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN
170256,2018,202,89.0,8.232880e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN


In [69]:
train = ratings[ratings['diff'].notnull()]
train.reset_index(drop=True, inplace=True)
train

,year,country_id,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,16.0,6.321578e-07,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
1,2013,0,19.0,1.799218e-06,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,1.0
2,2013,0,21.0,6.905108e-06,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
3,2013,0,27.0,2.577259e-06,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,0.0
4,2013,0,28.0,1.458826e-06,0.017864,0.035276,1.000000,1.026266,1.327406,0.162662,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
77731,2021,202,85.0,3.288542e-05,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0
77732,2021,202,87.0,4.543334e-05,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0
77733,2021,202,90.0,1.134952e-05,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,0.0
77734,2021,202,94.0,3.524696e-08,0.318998,0.020668,1.045755,0.951551,1.236016,0.162160,0.502048,1.0


In [70]:
# rating에서 diff가 null이 아닌 것들만 추출
test = ratings[ratings['diff'].isnull()]
test.reset_index(drop=True, inplace=True)
test

,year,country_id,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,4.862752e-08,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,NaN
1,2013,0,20.0,4.862752e-08,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,NaN
2,2013,0,22.0,2.676945e-04,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,NaN
3,2013,0,24.0,2.888475e-04,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,NaN
4,2013,0,39.0,3.929104e-05,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
92517,2018,202,80.0,7.380276e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN
92518,2018,202,86.0,6.606483e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN
92519,2018,202,88.0,2.919775e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN
92520,2018,202,89.0,8.232880e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN


In [71]:
# data 합치기
ratings = pd.concat([train, test])
ratings.reset_index(drop=True, inplace=True)
ratings

,year,country_id,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,16.0,6.321578e-07,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.0
1,2013,0,19.0,1.799218e-06,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,1.0
2,2013,0,21.0,6.905108e-06,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.0
3,2013,0,27.0,2.577259e-06,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.0
4,2013,0,28.0,1.458826e-06,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
170253,2018,202,80.0,7.380276e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN
170254,2018,202,86.0,6.606483e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN
170255,2018,202,88.0,2.919775e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN
170256,2018,202,89.0,8.232880e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,NaN


# encoding

In [72]:
# year encoding
year_dict = {}
for i in set(ratings['year']):
    year_dict[i] = len(year_dict)
n_year = len(year_dict)                  

# Country encoding
country_dict = {}
start_point = n_year                     
for i in set(ratings['country_id']):
    country_dict[i] = len(country_dict)
n_country = len(country_dict)            
start_point += n_country                 

# Item encoding
item_dict = {}
start_point = n_country                  
for i in set(ratings['item_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)                  
start_point += n_item                    

# preference encoding
pre_index = start_point
start_point += 1

# GDP_growth encoding   
gdp_index = start_point
start_point += 1

# pop_growth encoding
pop_index = start_point
start_point += 1

# 환율 4종
USD_index = start_point
start_point += 1

JPY_index = start_point
start_point += 1

EUR_index = start_point
start_point += 1

CNY_index = start_point
start_point += 1

# 유가
WTI_index = start_point
start_point += 1

num_x = start_point                     # 전체 변수의 숫자 (년도 + 국가 개수 + 품목 개수 + GDP_growth + pop_growth, preference) 총 311개

In [73]:
x = ratings

In [74]:
# Generate X data
data = []
y = []
for i in range(len(x)):
    case = x.iloc[i]
    x_index = []
    x_value = []
    x_index.append(year_dict[case['year']])        # year id one-hot encoding
    x_value.append(1.)                             # 있으면 1
    x_index.append(country_dict[case['country_id']])  # country id one-hot encoding
    x_value.append(1.)
    x_index.append(item_dict[case['item_id']])     # item id one-hot encoding
    x_value.append(1.)
    x_index.append(pre_index)                      # preference
    x_value.append(case['preference'])
    x_index.append(gdp_index)                      
    x_value.append(case['GDP_growth'])
    x_index.append(pop_index)
    x_value.append(case['pop_growth'])
    x_index.append(USD_index)
    x_value.append(case['USD'])
    x_index.append(JPY_index)
    x_value.append(case['JPY'])
    x_index.append(EUR_index)
    x_value.append(case['EUR'])
    x_index.append(CNY_index)
    x_value.append(case['CNY'])
    x_index.append(WTI_index)
    x_value.append(case['WTI'])
    
    
    data.append([x_index, x_value])                # 만들어진 한줄의 데이터를([[year_index, country_index, item_index, gdp_index, pop_index], [1, 1, 1, 1, 1]] 형태) data에 저장
    y.append(case['diff'])                         # 종속변수
    if (i % 30000) == 0:
        print('Encoding ', i, ' cases...')

Encoding  0  cases...
Encoding  30000  cases...
Encoding  60000  cases...
Encoding  90000  cases...
Encoding  120000  cases...
Encoding  150000  cases...


In [75]:
idx = ratings[ratings['diff'].notnull()].index[-1]
idx

77735

In [76]:
trainset = data[:idx+1]
testset = data[idx+1:]
y = y[:idx+1]

In [77]:
from IPython.display import display
print(len(trainset))
print(len(testset))
print(len(y))

77736
92522
77736


# model

In [78]:
class FactorizationMachine():
    def __init__(self, N, K, data, y, test_data, lr, l2_reg, l2_lambda, epoch, early_stop_window, train_ratio=0.75):
        """
        :param k: number of latent vector
        :param lr: learning rate
        :param l2_reg: bool parameter for L2 regularization
        :param l2_lambda: lambda of L2 regularization
        :param epoch: training epoch
        """
        self._K = K
        self._N = N
        self._data = data
        self._y = y
        self._test_data = test_data
        self._lr = lr
        self._l2_reg = l2_reg
        self._l2_lambda = l2_lambda
        self._epoch = epoch
        self._early_stop_window = early_stop_window
        self._valid_loss_list = []
        # Train/Valid/Test 분리
        cutoff = int(train_ratio * len(data))
        self.train_x = data[:cutoff]
        self.valid_x = data[cutoff:]
        self.train_y = y[:cutoff]
        self.valid_y = y[cutoff:]
        self.test_x = test_data

    def _load_dataset(self):
        # init FM vectors
        self._init_vectors()
        print("Finish init FM vectors.")

    def _init_vectors(self):
        # w 초기화
        self.w = np.random.normal(scale=1./self._N, size=(self._N))          # 사이즈는 변수의 수 만큼
        # v 초기화
        self.v = np.random.normal(scale=1./self._K, size=(self._N, self._K))  # 사이즈는 (변수의 수 X K)
        self.train_x_data = []
        self.train_y_data = np.zeros((len(self.train_x)))
        self.valid_x_data = []
        self.valid_y_data = np.zeros((len(self.valid_x)))

        self.train_y_data = np.array(self.train_y)
        for n, row in enumerate(self.train_x):
            self.train_x_data.append([np.array(row[0]), np.array(row[1])])
        
        self.valid_y_data = np.array(self.valid_y)
        for n, row in enumerate(self.valid_x):
            self.valid_x_data.append([np.array(row[0]), np.array(row[1])])

    def train(self):
        """
        Train FM model by Gradient Descent with L2 regularization
        """
        self._load_dataset()
        for epoch_num in range(1, self._epoch):
            train_y_hat = self.predict(data=self.train_x_data)
            valid_y_hat = self.predict(data=self.valid_x_data)
            train_loss = self._get_loss(y_data=self.train_y_data, y_hat=train_y_hat)
            valid_loss = self._get_loss(y_data=self.valid_y_data, y_hat=valid_y_hat)
            train_auc = roc_auc_score(self.train_y_data, train_y_hat)
            valid_auc = roc_auc_score(self.valid_y_data, valid_y_hat)
            self._print_learning_info(epoch=epoch_num, train_loss=train_loss, valid_loss=valid_loss,
                                      train_auc=train_auc, valid_auc=valid_auc)
            if self._check_early_stop(valid_loss=valid_loss):
                print("Early stop at epoch:", epoch_num)
                return 0

            self._stochastic_gradient_descent(self.train_x_data, self.train_y_data)

    def predict(self, data):
        """
        Implementation of FM model's equation on O(kmd)

        -----
        Numpy array shape : (n, [index of md], [value of md])
        md : none-zero feature
        """  
        num_data = len(data)
        scores = np.zeros(num_data)
        for n in range(num_data):
            feat_idx = data[n][0]
            val = data[n][1]

            # linear feature score
            linear_feature_score = np.sum(self.w[feat_idx] * val)

            # factorized feature score
            vx = self.v[feat_idx] * (val.reshape(-1, 1))
            cross_sum = np.sum(vx, axis=0)
            square_sum = np.sum(vx * vx, axis=0)
            cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

            # Model's equation
            scores[n] = linear_feature_score + cross_feature_score

        # Sigmoid transformation for binary classification
        scores = 1.0 / (1.0 + np.exp(-scores))
        return scores

    def inference(self, data):
        
        self.test_x_data = []
        for n, row in enumerate(self.test_x):
            self.test_x_data.append([np.array(row[0]), np.array(row[1])])

        num_data = len(data)
        scores = np.zeros(num_data)
        for n in range(num_data):
            feat_idx = data[n][0]
            val = np.array(data[n][1])

            # linear feature score
            linear_feature_score = np.sum(self.w[feat_idx] * val)

            # factorized feature score
            vx = self.v[feat_idx] * (val.reshape(-1, 1))
            cross_sum = np.sum(vx, axis=0)
            square_sum = np.sum(vx * vx, axis=0)
            cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

            # Model's equation
            scores[n] = linear_feature_score + cross_feature_score

        # Sigmoid transformation for binary classification
        scores = 1.0 / (1.0 + np.exp(-scores))
        print(scores)
        return scores

    def _get_loss(self, y_data, y_hat):
        """
        Calculate loss with L2 regularization (two type of coeficient - w,v)
        """
        l2_norm = 0
        if self._l2_reg:
            w_norm = np.sqrt(np.sum(np.square(self.w)))
            v_norm = np.sqrt(np.sum(np.square(self.v)))
            l2_norm = self._l2_lambda * (w_norm + v_norm)
        return -1 * np.sum( (y_data * np.log(y_hat)) + ((1 - y_data) * np.log(1 - y_hat)) ) + l2_norm

    def _check_early_stop(self, valid_loss):
        self._valid_loss_list.append(valid_loss)
        if len(self._valid_loss_list) > 5:
            prev_loss = self._valid_loss_list[len(self._valid_loss_list) - self._early_stop_window]
            curr_loss = valid_loss
            if prev_loss < curr_loss:
                return True
        return False

    def _print_learning_info(self, epoch, train_loss, valid_loss, train_auc, valid_auc):
        print("epoch:", epoch, "||", "train_loss:", train_loss, "||", "valid_loss:", valid_loss,
              "||", "Train AUC:", train_auc, "||", "Test AUC:", valid_auc)

    def _stochastic_gradient_descent(self, x_data, y_data):
        """
        Update each coefs (w, v) by Gradient Descent
        """
        for data, y in zip(x_data, y_data):
            feat_idx = data[0]
            val = data[1]
            vx = self.v[feat_idx] * (val.reshape(-1, 1))

            # linear feature score
            linear_feature_score = np.sum(self.w[feat_idx] * val)

            # factorized feature score
            vx = self.v[feat_idx] * (val.reshape(-1, 1))
            cross_sum = np.sum(vx, axis=0)
            square_sum = np.sum(vx * vx, axis=0)
            cross_feature_score = 0.5 * np.sum(np.square(cross_sum) - square_sum)

            # Model's equation
            score = linear_feature_score + cross_feature_score
            y_hat = 1.0 / (1.0 + np.exp(-score))
            cost = y_hat - y

            if self._l2_reg:
                self.w[feat_idx] = self.w[feat_idx] - cost * self._lr * (val + self._l2_lambda * self.w[feat_idx])
                self.v[feat_idx] = self.v[feat_idx] - cost * self._lr * ((sum(vx) * (val.reshape(-1, 1)) - (vx * (val.reshape(-1, 1)))) + self._l2_lambda * self.v[feat_idx])
            else:
                self.w[feat_idx] = self.w[feat_idx] - cost * self._lr * val
                self.v[feat_idx] = self.v[feat_idx] - cost * self._lr * (sum(vx) * (val.reshape(-1, 1)) - (vx * (val.reshape(-1, 1))))

In [79]:
if __name__ == "__main__":
    fm = FactorizationMachine(N = num_x,
                              K=4,
                              data = trainset,
                              y = y,
                              test_data = testset,
                              lr=0.001,
                              l2_reg=True,
                              l2_lambda=0.0001,
                              epoch=200,
                              early_stop_window=3)

    fm.train()
    predict_proba = fm.inference(data=testset)

Finish init FM vectors.
epoch: 1 || train_loss: 41820.696415464605 || valid_loss: 13906.837191400917 || Train AUC: 0.5081511194157272 || Test AUC: 0.5133329976363137
epoch: 2 || train_loss: 40439.74189660636 || valid_loss: 13677.936972460888 || Train AUC: 0.5295089445629977 || Test AUC: 0.49499951639423545
epoch: 3 || train_loss: 40185.13057592777 || valid_loss: 13597.834950267277 || Train AUC: 0.5517535766358721 || Test AUC: 0.5123048714547165
epoch: 4 || train_loss: 40017.4933845745 || valid_loss: 13543.134712182135 || Train AUC: 0.565406490757752 || Test AUC: 0.5249734708074664
epoch: 5 || train_loss: 39901.13595009935 || valid_loss: 13503.919381266925 || Train AUC: 0.5735945982418216 || Test AUC: 0.5339369109029093
epoch: 6 || train_loss: 39815.79073337654 || valid_loss: 13474.40788208589 || Train AUC: 0.5790241266159833 || Test AUC: 0.5403491351804929
epoch: 7 || train_loss: 39750.06946366873 || valid_loss: 13451.38999204073 || Train AUC: 0.5829577042362883 || Test AUC: 0.54505960

In [107]:
predict_proba

array([0.53348703, 0.60320542, 0.68590298, ..., 0.68521447, 0.46704613,
       0.58074657])

# predict_proba

In [108]:
a = list(predict_proba)

In [109]:
test

,year,country_id,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,4.862752e-08,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.533487
1,2013,0,20.0,4.862752e-08,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.603205
2,2013,0,22.0,2.676945e-04,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.685903
3,2013,0,24.0,2.888475e-04,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.669035
4,2013,0,39.0,3.929104e-05,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.596791
...,...,...,...,...,...,...,...,...,...,...,...,...
92517,2018,202,80.0,7.380276e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.536316
92518,2018,202,86.0,6.606483e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.681047
92519,2018,202,88.0,2.919775e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.685214
92520,2018,202,89.0,8.232880e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.467046


In [110]:
len(a)

92522

In [111]:
test['diff'] = a
test

/var/folders/bz/yk1btxt17y920gj06j1__cy00000gn/T/ipykernel_33589/2414523319.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['diff'] = a


,year,country_id,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,4.862752e-08,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.533487
1,2013,0,20.0,4.862752e-08,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.603205
2,2013,0,22.0,2.676945e-04,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.685903
3,2013,0,24.0,2.888475e-04,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.669035
4,2013,0,39.0,3.929104e-05,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.596791
...,...,...,...,...,...,...,...,...,...,...,...,...
92517,2018,202,80.0,7.380276e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.536316
92518,2018,202,86.0,6.606483e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.681047
92519,2018,202,88.0,2.919775e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.685214
92520,2018,202,89.0,8.232880e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.467046


In [112]:
test.to_csv('./processed/CFFM_predict_proba_13_18_WTI_FX.csv',index = False)

# FM precision

In [113]:
# pickle 파일 불러오기
result_13_18 = pd.read_csv('./processed/CFFM_predict_proba_13_18_WTI_FX.csv')
with open('./processed/country_to_idx.pkl', 'rb') as f:
    country_to_idx = pickle.load(f)
with open('./processed/idx_to_country.pkl', 'rb') as f:
    idx_to_country = pickle.load(f)
with open('./processed/new_exp_3years.pkl', 'rb') as f:
    new_exp_3years = pickle.load(f)
with open('./processed/new_exp_5years.pkl', 'rb') as f:
    new_exp_5years = pickle.load(f)

In [114]:
result_13_18

,year,country_id,item_id,preference,GDP_growth,pop_growth,USD,JPY,EUR,CNY,WTI,diff
0,2013,0,10.0,4.862752e-08,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.533487
1,2013,0,20.0,4.862752e-08,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.603205
2,2013,0,22.0,2.676945e-04,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.685903
3,2013,0,24.0,2.888475e-04,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.669035
4,2013,0,39.0,3.929104e-05,0.017864,0.035276,1.00000,1.026266,1.327406,0.162662,1.000000,0.596791
...,...,...,...,...,...,...,...,...,...,...,...,...
92517,2018,202,80.0,7.380276e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.536316
92518,2018,202,86.0,6.606483e-07,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.681047
92519,2018,202,88.0,2.919775e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.685214
92520,2018,202,89.0,8.232880e-06,0.194652,0.232127,1.00558,0.910591,1.186927,0.152041,0.439715,0.467046


In [115]:
from tqdm import tqdm 

In [119]:
# precision 함수로 만들기
def FM_precision(country, current_year, new_exp_Nyears, proba_threshold, neighbor_size):
    c_items = new_exp_Nyears[(idx_to_country[country], current_year)]
    rec_13_18 = result_13_18[result_13_18['diff'] > proba_threshold]
    rec_items = rec_13_18[rec_13_18['country_id']==country].query('year==@current_year').item_id.tolist()
    hit_items = [i for i in c_items if i in rec_items]
    if len(rec_items) == 0:
        precision = 0
    else:
        precision = len(hit_items) / len(rec_items)
    return precision

# 해당년도 모든 나라에 대해서 precision 계산
def FM_precision_year(year, new_exp_Nyears, proba_threshold=0.5, neighbor_size=16):
    precision = []
    c_len = len(idx_to_country)
    for country in range(0, c_len):
        precision.append(FM_precision(country, year, new_exp_Nyears, proba_threshold, neighbor_size))
    return sum(precision) / c_len

# 전체 데이터에 대해서 precision 계산
def FM_precision_all(n_year, proba_threshold=0.5, neighbor_size=16):
    precision = []
    c_len = len(idx_to_country)
    if n_year == 3:
        years = range(2013, 2019)
        for country in range(0, c_len):
            for year in years:
                precision.append(FM_precision(country, year, new_exp_3years, proba_threshold, neighbor_size))
    elif n_year == 5:
        for country in range(0, c_len):
            years = range(2013, 2017)
            for year in years:
                precision.append(FM_precision(country, year, new_exp_5years, proba_threshold, neighbor_size))
    return sum(precision) / (c_len*len(years))

In [120]:
def get_best_combination(n_year):
    best_score = 0
    best_thres = 0
    for thres in tqdm(np.arange(0.1,0.9, 0.01)):
        score = FM_precision_all(n_year, thres)
        if score >= best_score:
            best_score = score
            best_thres = thres
    return best_score, best_thres

In [121]:
# 2018년도 모든 나라에 대해서 향후 3년 내의 precision 계산
FM_precision_year(2018, new_exp_3years)

0.3671619434206606

In [122]:
# 전체 연도 모든 나라에 대해서 향후 3년 내의 precision 계산
FM_precision_all(3, 0.5)

0.38959365627586584

In [123]:
get_best_combination(3)

100%|██████████| 80/80 [01:52<00:00,  1.41s/it]


(0.3956010662108777, 0.5199999999999998)

In [124]:
# 2016년도 모든 나라에 대해서 향후 5년 내의 precision 계산
FM_precision_year(2016, new_exp_5years)

0.48303240657239915

In [125]:
# 전체 연도 모든 나라에 대해서 향후 5년 내의 precision 계산
FM_precision_all(5, 0.5)

0.47139362847336086

In [126]:
get_best_combination(5)

100%|██████████| 80/80 [01:14<00:00,  1.07it/s]


(0.4855507469674861, 0.5399999999999998)